<a href="https://colab.research.google.com/github/jgalazka/ERCC_analysis/blob/main/GLDS-235-Liver/GLDS_235_Liver_ERCC_02012021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GeneLab ERCC spike-in analysis notebook

This notebook contains an analysis of RNA-seq counts data generated from ERCC spike-ins.

Here GLDS-235 Liver is analyzed


## Section 1: Setting up the notebook

In [ ]:
# Set-up notebook to allow inline running of R code
# Having done this, any code block starting with %RR will be run as R
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [28]:
# import python packages
import pandas as pd
from urllib.request import urlopen, quote, urlretrieve
from json import loads
from re import search
import zipfile

In [ ]:
# Install R libraries necessary
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("erccdashboard")

In [ ]:
# Load R libraries
%%R
library(erccdashboard)

In [ ]:
# Code to pull metadata zip from GeneLab
# Credit to Kirill Grigorev
GENELAB_ROOT = "https://genelab-data.ndc.nasa.gov"
GLDS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/data/"
FILELISTINGS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/filelistings/"
ISA_ZIP_REGEX = r'.*_metadata_.*[_-]ISA\.zip$'

def read_json(url):
    with urlopen(url) as response:
        return loads(response.read().decode())

def get_isa(accession):
    glds_json = read_json(GLDS_URL_PREFIX + accession)
    try:
        _id = glds_json[0]["_id"]
    except (AssertionError, TypeError, KeyError, IndexError):
        raise ValueError("Malformed JSON?")
    isa_entries = [
        entry for entry in read_json(FILELISTINGS_URL_PREFIX + _id)
        if search(ISA_ZIP_REGEX, entry["file_name"])
    ]
    if len(isa_entries) == 0:
        raise ValueError("Unexpected: no ISAs found")
    elif len(isa_entries) > 1:
        raise ValueError("Unexpected: multiple files match the ISA regex")
    else:
        entry = isa_entries[0]
        version = entry["version"]
        url = GENELAB_ROOT + entry["remote_url"] + "?version={}".format(version)
        alt_url = (
            GENELAB_ROOT + "/genelab/static/media/dataset/" +
            quote(entry["file_name"]) + "?version={}".format(version)
        )
        return entry["file_name"], version, url, alt_url

## Parse metadata
Get ISA file containing metadata and get sample and assay tables.
Combine these on sample name

In [38]:
accession = 'GLDS-48' # Change this as necessary
isaurl = get_isa(accession)[3]
filehandle, _ = urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
zip_file_object.namelist() # Print contents of zip file. Pick relevant one from list

['.DS_Store',
 's_RR1 Spaceflight.txt',
 'a_rodent_research-1_(rr1)_nasa_liver_transcription_profiling_RNA_Sequencing_(RNA-Seq).txt',
 'a_rodent_research-1_(rr1)_nasa_liver_histology_imaging.txt',
 'a__protein_expression_profiling_mass_spectrometry.txt',
 'i_Investigation.txt',
 'a_rr1-nasa_RNA_methylation_profiling_nucleotide_sequencing.txt',
 'a__DNA_methylation_profiling_nucleotide_sequencing.txt']

In [46]:
sample_file = zip_file_object.namelist()[1]
file = zip_file_object.open(sample_file)
sample_table = pd.read_csv(zip_file_object.open(sample_file), sep='\t')

assay_file = zip_file_object.namelist()[2]
file = zip_file_object.open(assay_file)
assay_table = pd.read_csv(zip_file_object.open(assay_file), sep='\t')

In [47]:
sample_table.head()

,Source Name,Protocol REF,Characteristics[organism],Term Source REF,Term Accession Number,Characteristics[strain],Term Source REF.1,Term Accession Number.1,Comment[Animal Source],"Characteristics[sex,http://purl.obolibrary.org/obo/PATO_0000047,EFO]",Term Source REF.2,Term Accession Number.2,Characteristics[Age at Launch],Unit,Term Source REF.3,Term Accession Number.3,"Characteristics[diet,http://www.ebi.ac.uk/efo/EFO_0002755,EFO]",Term Source REF.4,Term Accession Number.4,Comment[Feeding Schedule],Factor Value[Spaceflight],Term Source REF.5,Term Accession Number.5,Factor Value[Dissection Condition],Term Source REF.6,Term Accession Number.6,"Parameter Value[Light Cycle,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C90419,NCIT]",Term Source REF.7,Term Accession Number.7,"Parameter Value[habitat,http://purl.obolibrary.org/obo/ENVO_01000739,ENVO]",Term Source REF.8,Term Accession Number.8,"Parameter Value[duration,http://www.ebi.ac.uk/efo/EFO_0000433,EFO]",Unit.1,Term Source REF.9,Term Accession Number.9,Protocol REF.1,Parameter Value[Euthanasia Chemical],Term Source REF.10,Term Accession Number.10,Material Type,Term Source REF.11,Term Accession Number.11,Parameter Value[Carcass Preservation Method],Term Source REF.12,Term Accession Number.12,Parameter Value[Sample Preservation Method],Term Source REF.13,Term Accession Number.13,Parameter Value[Sample Storage Temperature],Unit.2,Term Source REF.14,Term Accession Number.14,Comment[LSDA Biospecimen ID],Comment[LSDA Biospecimen Subject ID],Sample Name,Comment[Original Submitted Sample Name],Comment[Sample Name Description]
0,RR1_GC_M31,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Ground Control,NaN,NaN,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http://purl.bioontology.org/ontology/MESH/D008099,Mini Cold Bag to -80C freezer,NaN,NaN,Mini Cold Bag to -80C freezer,NaN,NaN,-80,degree Celsius,UO,http://purl.obolibrary.org/obo/UO_0000027,5422/5427,445.0,Mmus_C57-6J_LVR_GC_I_Rep1_M31,RR1_GC_M31_LVR,Ground Control_Mouse31_Liver
1,RR1_GC_M32,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Ground Control,NaN,NaN,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http://purl.bioontology.org/ontology/MESH/D008099,Mini Cold Bag to -80C freezer,NaN,NaN,Mini Cold Bag to -80C freezer,NaN,NaN,-80,degree Celsius,UO,http://purl.obolibrary.org/obo/UO_0000027,5422/5427,445.0,Mmus_C57-6J_LVR_GC_I_Rep2_M32,RR1_GC_M32_LVR,Ground Control_Mouse32_Liver
2,RR1_FLT_M21,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Space Flight,MESH,http://purl.bioontology.org/ontology/MESH/D013026,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http:

In [48]:
assay_table.head()

,Sample Name,Protocol REF,Parameter Value[QA Instrument],Parameter Value[QA Assay],Parameter Value[QA Score],Unit,Term Source REF,Term Accession Number,Extract Name,Protocol REF.1,Parameter Value[library selection],Parameter Value[stranded],Parameter Value[library layout],Parameter Value[Library QA Instrument],Term Source REF.1,Term Accession Number.1,Protocol REF.2,Parameter Value[sequencing instrument],"Parameter Value[Read Length,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C153362,NCIT]",Unit.1,Term Source REF.2,Term Accession Number.2,Assay Name,Raw Data File,Protocol REF.3,Parameter Value[Merged Sequence Data File],Term Source REF.3,Term Accession Number.3,Parameter Value[rRNA Contamination],Unit.2,Term Source REF.4,Term Accession Number.4,"Parameter Value[Read Depth,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C155320,NCIT]",Unit.3,Term Source REF.5,Term Accession Number.5,Parameter Value[Merged Sequence Data Fastqc File],Term Source REF.6,Term Accession Number.6,Parameter Value[Merged Sequence Data Multiqc File],Term Source REF.7,Term Accession Number.7,"Parameter Value[trimmed sequence data,http://purl.obolibrary.org/obo/OBI_0002569,OBI]",Term Source REF.8,Term Accession Number.8,Parameter Value[Trimmed Sequence Data Fastqc File],Term Source REF.9,Term Accession Number.9,Parameter Value[Trimmed Sequence Data Multiqc File],Term Source REF.10,Term Accession Number.10,Parameter Value[Trimmed Report],Term Source REF.11,Term Accession Number.11,"Parameter Value[aligned sequence data,http://purl.obolibrary.org/obo/OBI_0002580,OBI]",Term Source REF.12,Term Accession Number.12,Parameter Value[Alignment Logs],Term Source REF.13,Term Accession Number.13,Parameter Value[Raw Counts Data File],Term Source REF.14,Term Accession Number.14,Parameter Value[Normalized Counts Data File],Term Source REF.15,Term Accession Number.15,"Parameter Value[differential expression analysis data transformation,http://purl.obolibrary.org/obo/OBI_0000650,OBI]",Term Source REF.16,Term Accession Number.16
0,Mmus_C57-6J_LVR_GC_I_Rep1_M31,nucleic acid extraction,Agilent 2100 Bioanalyzer,RNA 6000 Nano Kit,9.3,RNA Integrity Number,NCIT,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,RR1_GC_M31_LVR_RNA,RNA sequencing library construction,mRNA enrichment,STRANDED,SINGLE,Agilent 2100 Bioanalyzer,NaN,NaN,nucleic acid sequencing,Illumina HiSeq 3000,50,base pair,UO,http://purl.obolibrary.org/obo/UO_0000244,rna-seq,GLDS-48_transcriptomics_N-GC-31.tar.gz,GeneLab RNAseq data processing protocol,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,5.05,percent,UO,http://purl.obolibrary.org/obo/UO_0000187,36442893,read,SO,http://purl.obolibrary.org/obo/SO_0000150,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,"GLDS-48_rna_seq_raw_multiqc_report.html, GLDS-...",NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,"GLDS-48_rna_seq_trimmed_multiqc_report.html, G...",NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31....,NaN,NaN,GLDS-48_rna_seq_Normalized_Counts.csv,NaN,NaN,"GLDS-48_rna_seq_contrasts.csv, GLDS-48_rna_seq...",NaN,NaN
1,Mmus_C57-6J_LVR_GC_I_Rep2_M32,nucleic acid extraction,Agilent 2100 Bioanalyzer,RNA 6000 Nano Kit,9.8,RNA Integrity Number,NCIT,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,RR1_GC_M32_LVR_RNA,RNA sequencing library construction,mRNA enrichment,STRANDED,SINGLE,Agilent 2100 Bioanalyzer,NaN,NaN,nucleic acid sequencing,Illumina HiSeq 3000,50,base pair,UO,http://purl.obolibrary.org/obo/UO_0000244,rna-seq,GLDS-48_transcriptomics_N-GC-32.tar.gz,GeneLab RNAseq data processing protocol,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep2_M32_...,NaN,NaN,5.74,percent,UO,http://purl.obolibrary.org/obo/UO_0000187,38631142,read,SO,http://purl.obolibrary.org/obo/SO_0000150,GLDS-48_rna_seq

## Single Sample ERCC Analysis
Because samples may be combined by users outside of their original study, it is useful to have ERCC metrics that can be calculated from single samples. These could include: limit of detection, dynamic range, R^2 from measured vs expected plot, etc.

## Full Study ERCC Analysis
In some studies, Mix 1 and Mix 2 are added to samples in a way that allows the power of a study to be evaluated. For example, all flight samples may contain Mix 1 while all ground contain Mix 2. In this case, using the ERCC Dashboard package allows an assessment of DE analysis between flight and ground by using the know ratios of transcripts in the ERCC Mix 1 and Mix 2 standards.